In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score

In [3]:
url = 'https://raw.githubusercontent.com/Shimix77/Diplomova_praca/main/lemma_sk.csv'
lemma_sk = pd.read_csv(url, header = 0, delimiter = ";") #skiprows=2
lemma_sk.drop_duplicates(subset ="Konkretny_tvar_slova", keep = "first", inplace = True)
lemma_sk.shape

(1176800, 3)

In [4]:
lemma_sk.head(6)

,Zakladny_tvar_slova,Konkretny_tvar_slova,Znacka
0,A,A,W
1,AB,AB,W
2,ABS,ABS,W
3,AC,AC,W
4,AD,AD,W
5,ADOS,ADOS,W


In [5]:
url = "https://raw.githubusercontent.com/Shimix77/Diplomova_praca/main/stopwords_sk.csv"
stopwords_sk = pd.read_csv(url, header = 0) #skiprows=2
stopwords_sk.shape

(418, 1)

In [6]:
stopwords_sk.head(6)

,stopwords_slovak
0,a
1,aby
2,aj
3,ak
4,aká
5,akáže


In [7]:
train = pd.read_csv('datasource_ML.csv',delimiter=',')
#train = pd.read_csv('Datasource_pre_ML - 50vs50.csv',delimiter=',')
train.shape

(1177, 7)

In [8]:
train.head(6)

,Politik,Strana,Relácia,Výrok,Rok,Mesiac,Hodnota
0,Eduard Heger,Oľano,Sobotné dialógy,"Dvadsaťšesť ich nominantov (opozičných strán, ...",2022,2,Nepravda
1,Eduard Heger,Oľano,Sobotné dialógy,"Keď tu boli iné vlády, vlády, Smeru tak keď bo...",2022,2,Pravda
2,Eduard Heger,Oľano,Sobotné dialógy,Maďarsko tam má prítomnosť spojeneckých vojsk ...,2022,2,Pravda
3,Eduard Heger,Oľano,Sobotné dialógy,"Toto nie je jediná zmluva (DCA, pozn.), ktorá ...",2022,2,Pravda
4,Eduard Heger,Oľano,Sobotné dialógy,"My sme na to, aby táto TPS nerástla investoval...",2022,2,Pravda
5,Eduard Heger,Oľano,Sobotné dialógy,"Ako viete, naša vláda vlastne zaviedla trinást...",2022,2,Zavádzajúce


In [9]:
#nastavenie datasetu na malé písmená
def lower_case(df):
    df['Výrok'] = df['Výrok'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    print(df['Výrok'].head())
    
lower_case(train)

0    dvadsaťšesť ich nominantov (opozičných strán, ...
1    keď tu boli iné vlády, vlády, smeru tak keď bo...
2    maďarsko tam má prítomnosť spojeneckých vojsk ...
3    toto nie je jediná zmluva (dca, pozn.), ktorá ...
4    my sme na to, aby táto tps nerástla investoval...
Name: Výrok, dtype: object


In [10]:
#očistenie textu
def cleaning_text(vyrok):
    vyroky = " ".join(filter(lambda x: x[0]!= '!' , vyrok.split()))
    #vyroky = re.sub(r'[^a-zA-Z]', ' ', vyroky)
    vyroky = re.sub(r'[^\w\s]', ' ', vyroky)
    vyroky = vyroky.split()
    vyroky = " ".join(vyroky)
    return vyroky

train['Výrok_opravený'] = train.Výrok.apply(cleaning_text)
train.head(6)

,Politik,Strana,Relácia,Výrok,Rok,Mesiac,Hodnota,Výrok_opravený
0,Eduard Heger,Oľano,Sobotné dialógy,"dvadsaťšesť ich nominantov (opozičných strán, ...",2022,2,Nepravda,dvadsaťšesť ich nominantov opozičných strán po...
1,Eduard Heger,Oľano,Sobotné dialógy,"keď tu boli iné vlády, vlády, smeru tak keď bo...",2022,2,Pravda,keď tu boli iné vlády vlády smeru tak keď bol ...
2,Eduard Heger,Oľano,Sobotné dialógy,maďarsko tam má prítomnosť spojeneckých vojsk ...,2022,2,Pravda,maďarsko tam má prítomnosť spojeneckých vojsk ...
3,Eduard Heger,Oľano,Sobotné dialógy,"toto nie je jediná zmluva (dca, pozn.), ktorá ...",2022,2,Pravda,toto nie je jediná zmluva dca pozn ktorá tu te...
4,Eduard Heger,Oľano,Sobotné dialógy,"my sme na to, aby táto tps nerástla investoval...",2022,2,Pravda,my sme na to aby táto tps nerástla investovali...
5,Eduard Heger,Oľano,Sobotné dialógy,"ako viete, naša vláda vlastne zaviedla trinást...",2022,2,Zavádzajúce,ako viete naša vláda vlastne zaviedla trinásty...


In [11]:
train_lemmatize_stopwords = train.Výrok_opravený.str.split()
list_of_konkretny_tvar = lemma_sk["Konkretny_tvar_slova"].to_list()
list_of_stopwords = stopwords_sk["stopwords_slovak"].to_list()
ListOfMyList = []

#def lemmatizing_stopwords_text():
a = 1177
for vyroky in train_lemmatize_stopwords:
    myList = []
    for word in vyroky:
        if word not in list_of_stopwords:
            if word in list_of_konkretny_tvar:
                myList.append(lemma_sk.loc[lemma_sk["Konkretny_tvar_slova"]==word,"Zakladny_tvar_slova"].iloc[0])
            else:
                myList.append(word)  
    myList = " ".join(myList)
    ListOfMyList.append(myList)

In [12]:
train['Vyrok_lematizovany_stopwords'] = pd.Series(ListOfMyList)
train = train.drop(['Výrok'], axis=1)
train = train.drop(['Výrok_opravený'], axis=1)
train.head(3)

,Politik,Strana,Relácia,Rok,Mesiac,Hodnota,Vyrok_lematizovany_stopwords
0,Eduard Heger,Oľano,Sobotné dialógy,2022,2,Nepravda,dvadsaťšesť nominant opozičný strana pozn priz...
1,Eduard Heger,Oľano,Sobotné dialógy,2022,2,Pravda,vláda vláda smer pokles cena nepreniesť občan ...
2,Eduard Heger,Oľano,Sobotné dialógy,2022,2,Pravda,maďarsko prítomnosť spojenecký vojsko rok samo...


In [13]:
train = train.rename({'Vyrok_lematizovany_stopwords': 'Výrok'}, axis=1)

In [14]:
#Odstránenie diakritiky

def cleaning_text(vyroky):
    vyroky = re.sub(r'[àáâãäå]', 'a', vyroky)
    vyroky = re.sub(r'[èéêë]', 'e', vyroky)
    vyroky = re.sub(r'[ìíîï]', 'i', vyroky)
    vyroky = re.sub(r'[òóôõö]', 'o', vyroky)
    vyroky = re.sub(r'[ùúûü]', 'u', vyroky)
    vyroky = re.sub(r'[ý]', 'y', vyroky)
    vyroky = re.sub(r'[ť]', 't', vyroky)
    vyroky = re.sub(r'[š]', 's', vyroky)
    vyroky = re.sub(r'[ď]', 'd', vyroky)
    vyroky = re.sub(r'[ž]', 'z', vyroky)
    vyroky = re.sub(r'[č]', 'c', vyroky)
    vyroky = re.sub(r'[ň]', 'n', vyroky)
    vyroky = re.sub(r'[ľ]', 'l', vyroky)
    vyroky = vyroky.split()
    vyroky = " ".join(vyroky)
    return vyroky 

train['Výrok'] = train.Výrok.apply(cleaning_text)

#Indexácia hodnôt
Hodnota_index={"Pravda": 1,"Zavádzajúce": 0,"Nepravda": 0,"Neoveriteľné": 0}
#Hodnota_index={"Pravda": 1,"Zavádzajúce": 2,"Nepravda": 3,"Neoveriteľné": 4}
data = [train]

for dataset in data:
    dataset['Hodnota'] = dataset['Hodnota'].map(Hodnota_index)

train.head(6)

,Politik,Strana,Relácia,Rok,Mesiac,Hodnota,Výrok
0,Eduard Heger,Oľano,Sobotné dialógy,2022,2,0,dvadsatsest nominant opozicny strana pozn priz...
1,Eduard Heger,Oľano,Sobotné dialógy,2022,2,1,vlada vlada smer pokles cena nepreniest obcan ...
2,Eduard Heger,Oľano,Sobotné dialógy,2022,2,1,madarsko pritomnost spojenecky vojsko rok samo...
3,Eduard Heger,Oľano,Sobotné dialógy,2022,2,1,jediny zmluva dca pozn teraz svet existovat zm...
4,Eduard Heger,Oľano,Sobotné dialógy,2022,2,1,tps nerast investovat rok 2020 160 000 000 vla...
5,Eduard Heger,Oľano,Sobotné dialógy,2022,2,0,vediet vlada vlastne zaviest trinasty dochodok...


In [15]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x)
vyrok_vectors = vectorizer.fit_transform(train['Výrok'])
vyrok_vectors = vyrok_vectors.toarray()

y = train["Hodnota"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(vyrok_vectors, y, test_size = 0.20, random_state = 0)

In [17]:
#Naive Bayes
#classifier_np = GaussianNB()
classifier_np = MultinomialNB()
classifier_np.fit(X_train, y_train)
y_pred_np = classifier_np.predict(X_test)
np_score = accuracy_score(y_test, y_pred_np)
#Decision Tree
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)
y_pred_dt = classifier_dt.predict(X_test)
dt_score = accuracy_score(y_test, y_pred_dt)
#Logistic Regression
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)
y_pred_lr = classifier_lr.predict(X_test)
lr_score = accuracy_score(y_test, y_pred_lr)
#Random Forest
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)
y_pred_rf = classifier_rf.predict(X_test)
rf_score = accuracy_score(y_test, y_pred_rf)
#Supoort Vector Machine
classifier_svm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
classifier_svm.fit(X_train, y_train)
y_pred_svm = classifier_svm.predict(X_test)
svm_score = accuracy_score(y_test, y_pred_svm)


print('Naive Bayes Accuracy: ', str(np_score*100))
print('Decision Tree Accuracy: ', str(dt_score*100))
print('Logistic Regression Accuracy: ', str(lr_score*100))
print('Random Forest Accuracy: ', str(rf_score*100))
print("Support Vector Machine Accuracy: ",str(svm_score*100))

Naive Bayes Accuracy:  72.03389830508475
Decision Tree Accuracy:  60.59322033898306
Logistic Regression Accuracy:  72.03389830508475
Random Forest Accuracy:  61.86440677966102
Support Vector Machine Accuracy:  72.03389830508475
